In [1]:
#https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/

#https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=&search_radius=50&search_lat=0&search_lng=0

In [2]:
import csv
import pandas as pd
import googlemaps
import requests 
import urllib.request
import time
from bs4 import BeautifulSoup
import pickle

In [2]:
my_list = [12, 65, 54, 39, 102, 339, 221, 50, 70, ]
  
# use anonymous function to filter and comparing 
# if divisible or not
result = list(filter(lambda x: (x % 13 == 0), my_list)) 
result

[65, 39, 221]

In [3]:
df2 = pickle.load(open('imm_p.pickle','rb'))
df2.head()

,Court,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry
0,Eloy,"[Cortes, John W., Gaz, Jennifer I., Habich, Pa...","Padilla, Victoria","1705 E. Hanna Road, Suite 366","Eloy, AZ 85131",520-466-3671,"[Eloy,, AZ, 85131]",32.755170,-111.553493,POINT (-111.55349 32.75517)
1,Florence,"[Guerrero, Marni, Taylor, Bruce A.]","Toliusis, Rachel",3260 N. Pinal Parkway Avenue,"Florence, AZ 85132",520-868-3341,"[Florence,, AZ, 85132]",33.031451,-111.387343,POINT (-111.38734 33.03145)
2,Phoenix,"[Bartlemay Sr., Robert C., Cottor, Elizabeth A...","Riebs, Sathya","250 N. Seventh Ave., Suite 300","Phoenix, AZ 85007",602-640-2747,"[Phoenix,, AZ, 85007]",33.448437,-112.074142,POINT (-112.07414 33.44844)
3,Tucson,"[Davis, John W., DeAngelis, Kathryn L., Keenan...","Graff, Corey","300 West Congress, Suite 300","Tucson, AZ 85701",520-670-5212,"[Tucson,, AZ, 85701]",32.222876,-110.974848,POINT (-110.97485 32.22288)
4,Adelanto,"[DePowell, Bryan E., Halperin, Ravit R., Marcu...","Roberts, Valerie (Acting)","10250 Rancho Road, Ste. 201A","Adelanto, CA 92301",760-561-6500,"[Adelanto,, CA, 92301]",34.582770,-117.409215,POINT (-117.40922 34.58277)


# tester

In [4]:
url = 'https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/'
response = requests.get(url)
response

<Response [200]>

In [5]:
soup = BeautifulSoup(response.text,'html.parser')

In [6]:
bio_test = soup.findAll('p')

bio_test[0:1]

[<p>Then-Acting Attorney General Matthew Whitaker appointed John W. Cortes to begin hearingcases in March 2019. Judge Cortes earned a Bachelor of Arts in 1999 from the City University ofNew York at Brooklyn College and a Juris Doctor in 2002 at the New York Law School. From2003 to 2018, he served as an assistant district attorney for the Suffolk County DistrictAttorneyâ€™s Office, and from 2013 to 2018, he served as a principal assistant district attorney forthe same court. Judge Cortes is a member of the State Bar of Arizona and New York State Bar.</p>]

# Get Court pages

In [7]:
# https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=eloy&search_radius=50&search_lat=0&search_lng=0

court_pages = []
for court in df2['Court']:
    url = 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location={}&search_radius=50&search_lat=0&search_lng=0'.format(court)
    court_pages.append(url)
court_pages[0:5]

['https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Eloy&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Florence&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Phoenix&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Tucson&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Adelanto&search_radius=50&search_lat=0&search_lng=0']

In [8]:

url = court_pages[0]
response = requests.get(url)
response
soup = BeautifulSoup(response.text,'html.parser')


# now judges by page

In [9]:
df2['Judges'].head()

0    [Cortes, John W., Gaz, Jennifer I., Habich, Pa...
1                  [Guerrero, Marni, Taylor, Bruce A.]
2    [Bartlemay Sr., Robert C., Cottor, Elizabeth A...
3    [Davis, John W., DeAngelis, Kathryn L., Keenan...
4    [DePowell, Bryan E., Halperin, Ravit R., Marcu...
Name: Judges, dtype: object

In [10]:
sample = df2['Judges'][0][3]

def judge_addy_part(judge):
    j_split = judge.split()
    res = [j.strip('.,') for j in j_split]
    name_num = len(res)
    # judge_hyphen =j_split
    if name_num == 1:
        judge_hyphen = res
    elif name_num == 2:
        judge_hyphen = res[1]+'-'+res[0]
    elif name_num == 3:
        judge_hyphen = res[1]+'-'+res[2]+'-'+res[0]
    else:
        judge_hyphen = res[2]+'-'+res[3]+'-'+res[0]+'-'+res[1]
    return judge_hyphen

judge_addy_part(df2['Judges'][0][3])


'Jose-L-Penalosa-Jr'

In [11]:
sample = df2['Judges'][0][3]

def judge_addy_list(list):
    j_list=[]
    for judge in list:
        j_split = judge.split()
        res = [j.strip('.,') for j in j_split]
        name_num = len(res)
        # judge_hyphen =j_split
        if name_num == 1:
            judge_hyphen = res
        elif name_num == 2:
            judge_hyphen = res[1]+'-'+res[0]
        elif name_num == 3:
            judge_hyphen = res[1]+'-'+res[2]+'-'+res[0]
        else:
            judge_hyphen = res[2]+'-'+res[3]+'-'+res[0]+'-'+res[1]
        j_list.append(judge_hyphen)
    return j_list

judge_addy_list(df2['Judges'][0])

['John-W-Cortes', 'Jennifer-I-Gaz', 'Paul-Habich', 'Jose-L-Penalosa-Jr']

In [12]:
judge_list_HY =[]
#https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/

for judge in df2['Judges'][0]:
    judge_list_HY.append(judge_addy_part(judge))
judge_list_HY
    
    

['John-W-Cortes', 'Jennifer-I-Gaz', 'Paul-Habich', 'Jose-L-Penalosa-Jr']

In [13]:
judge_addy_list(df2['Judges'][2])

['Robert-C-Bartlemay-Sr',
 'Elizabeth-A-Cottor',
 'LaMonte-S-Freerks',
 'Joseph-S-Imburgia',
 'Ken-Josephson',
 'Melissa-B-Karlen',
 'Munish-Sharda',
 'Linda-Spencer-Walters']

In [14]:
df2[45:52]

,Court,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry
46,Cleveland,"[Horton, Monte J., Imbacuan, Bruce D., Owens, ...","Roder, James W.",801 W. Superior Avenue,"Cleveland, OH 44113",216-802-1100,"[Cleveland,, OH, 44113]",41.498438,-81.668354,POINT (-81.66835 41.49844)
47,Portland,"[Hoeppner, Mindy E., Lemke, Kathy J., Zanfardi...","John, Michael","1220 SW 3rd Avenue, Suite 500","Portland, OR 97204",503-326-6341,"[Portland,, OR, 97204]",45.520247,-122.674195,POINT (-122.67419 45.52025)
48,Philadelphia,"[Carle, John B., Ehlers, Patrick J., Garcy, An...","Alicea, Alterra","900 Market Street, Suite 504","Philadelphia, PA 19107",215-656-7000,"[Philadelphia,, PA, 19107]",39.952724,-75.163526,POINT (-75.16353 39.95272)
50,Guaynabo\n (San Juan),"[Diaz-Rex, Julia, Espinal, Pedro J., Talavera-...","Quiles, Luis","#7 Tabonuco Street, Room 401","Guaynabo, PR 00968-4605",787-749-4386,"[Guaynabo,, PR, 00968-4605]",18.423681,-66.111922,POINT (-66.11192 18.42368)
51,Memphis,"[Holt, Rebecca L., Johnson, Kelly S., Maldonad...","Davis, J. Thomas","80 Monroe Ave, Suite 501","Memphis, TN 38103",901-528-5883,"[Memphis,, TN, 38103]",35.149022,-90.051629,POINT (-90.05163 35.14902)
52,Conroe,"[Brisack,Chris A., D'Andrea, Holly A., Evans, ...","Russelburg, Mark",806 Hilbig Road,"Conroe, TX 77301",936-520-5400,"[Conroe,, TX, 77301]",30.311877,-95.456051,POINT (-95.45605 30.31188)
53,Dallas,"[Davis-Gumbs, Xiomara D., Ferguson, Jason D., ...","Baker, Barbara T.","1100 Commerce Street, Suite 1060","Dallas, TX 75242",214-767-1814,"[Dallas,, TX, 75242]",32.776272,-96.796856,POINT (-96.79686 32.77627)


In [15]:
df2 = df2[df2.Court != 'York']
df2 = df2.reset_index(drop=True)
df2[45:52]

,Court,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry
45,Cleveland,"[Horton, Monte J., Imbacuan, Bruce D., Owens, ...","Roder, James W.",801 W. Superior Avenue,"Cleveland, OH 44113",216-802-1100,"[Cleveland,, OH, 44113]",41.498438,-81.668354,POINT (-81.66835 41.49844)
46,Portland,"[Hoeppner, Mindy E., Lemke, Kathy J., Zanfardi...","John, Michael","1220 SW 3rd Avenue, Suite 500","Portland, OR 97204",503-326-6341,"[Portland,, OR, 97204]",45.520247,-122.674195,POINT (-122.67419 45.52025)
47,Philadelphia,"[Carle, John B., Ehlers, Patrick J., Garcy, An...","Alicea, Alterra","900 Market Street, Suite 504","Philadelphia, PA 19107",215-656-7000,"[Philadelphia,, PA, 19107]",39.952724,-75.163526,POINT (-75.16353 39.95272)
48,Guaynabo\n (San Juan),"[Diaz-Rex, Julia, Espinal, Pedro J., Talavera-...","Quiles, Luis","#7 Tabonuco Street, Room 401","Guaynabo, PR 00968-4605",787-749-4386,"[Guaynabo,, PR, 00968-4605]",18.423681,-66.111922,POINT (-66.11192 18.42368)
49,Memphis,"[Holt, Rebecca L., Johnson, Kelly S., Maldonad...","Davis, J. Thomas","80 Monroe Ave, Suite 501","Memphis, TN 38103",901-528-5883,"[Memphis,, TN, 38103]",35.149022,-90.051629,POINT (-90.05163 35.14902)
50,Conroe,"[Brisack,Chris A., D'Andrea, Holly A., Evans, ...","Russelburg, Mark",806 Hilbig Road,"Conroe, TX 77301",936-520-5400,"[Conroe,, TX, 77301]",30.311877,-95.456051,POINT (-95.45605 30.31188)
51,Dallas,"[Davis-Gumbs, Xiomara D., Ferguson, Jason D., ...","Baker, Barbara T.","1100 Commerce Street, Suite 1060","Dallas, TX 75242",214-767-1814,"[Dallas,, TX, 75242]",32.776272,-96.796856,POINT (-96.79686 32.77627)


In [16]:
# df2['Judges_Hy'] = [df2['Judges'].apply(lambda x : [judge_addy_list(x) for x in df2['Judges']])]
Judges_Hy = []
count=0
for judges in df2['Judges']:
    Judges_Hy.append(judge_addy_list(judges))
df2['Judges_Hy'] = pd.Series(Judges_Hy)
df2.head()

,Court,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry,Judges_Hy
0,Eloy,"[Cortes, John W., Gaz, Jennifer I., Habich, Pa...","Padilla, Victoria","1705 E. Hanna Road, Suite 366","Eloy, AZ 85131",520-466-3671,"[Eloy,, AZ, 85131]",32.755170,-111.553493,POINT (-111.55349 32.75517),"[John-W-Cortes, Jennifer-I-Gaz, Paul-Habich, J..."
1,Florence,"[Guerrero, Marni, Taylor, Bruce A.]","Toliusis, Rachel",3260 N. Pinal Parkway Avenue,"Florence, AZ 85132",520-868-3341,"[Florence,, AZ, 85132]",33.031451,-111.387343,POINT (-111.38734 33.03145),"[Marni-Guerrero, Bruce-A-Taylor]"
2,Phoenix,"[Bartlemay Sr., Robert C., Cottor, Elizabeth A...","Riebs, Sathya","250 N. Seventh Ave., Suite 300","Phoenix, AZ 85007",602-640-2747,"[Phoenix,, AZ, 85007]",33.448437,-112.074142,POINT (-112.07414 33.44844),"[Robert-C-Bartlemay-Sr, Elizabeth-A-Cottor, La..."
3,Tucson,"[Davis, John W., DeAngelis, Kathryn L., Keenan...","Graff, Corey","300 West Congress, Suite 300","Tucson, AZ 85701",520-670-5212,"[Tucson,, AZ, 85701]",32.222876,-110.974848,POINT (-110.97485 32.22288),"[John-W-Davis, Kathryn-L-DeAngelis, Sean-H-Kee..."
4,Adelanto,"[DePowell, Bryan E., Halperin, Ravit R., Marcu...","Roberts, Valerie (Acting)","10250 Rancho Road, Ste. 201A","Adelanto, CA 92301",760-561-6500,"[Adelanto,, CA, 92301]",34.582770,-117.409215,POINT (-117.40922 34.58277),"[Bryan-E-DePowell, Ravit-R-Halperin, Steven-Ma..."


In [17]:
df2['CityZip'][0].split()[0].strip(',.')

'Eloy'

In [18]:
# good for pulling the city out of the cityzip feature.  now jsut need to hyphenate judges
judge_city =[]

for i in range(len(df2['Judges_Hy'])):
    for j in df2['Judges_Hy'][i]:
        next=[]
        hy_j = judge_addy_part(j)[0]
        city = df2['CityZip'][i]
        try:
            city = city.split()[0].strip(',.')
        except:
            pass
        # judge_hy = judge_addy_part(j)
        next.append(hy_j)
        next.append(city)
        judge_city.append(next)
judge_city[0:15]

[['John-W-Cortes', 'Eloy'],
 ['Jennifer-I-Gaz', 'Eloy'],
 ['Paul-Habich', 'Eloy'],
 ['Jose-L-Penalosa-Jr', 'Eloy'],
 ['Marni-Guerrero', 'Florence'],
 ['Bruce-A-Taylor', 'Florence'],
 ['Robert-C-Bartlemay-Sr', 'Phoenix'],
 ['Elizabeth-A-Cottor', 'Phoenix'],
 ['LaMonte-S-Freerks', 'Phoenix'],
 ['Joseph-S-Imburgia', 'Phoenix'],
 ['Ken-Josephson', 'Phoenix'],
 ['Melissa-B-Karlen', 'Phoenix'],
 ['Munish-Sharda', 'Phoenix'],
 ['Linda-Spencer-Walters', 'Phoenix'],
 ['John-W-Davis', 'Tucson']]

# now indy bios for judges

In [19]:
url3= 'https://www.ratemyimmigrationjudge.com/listing/hon-print-maggard-immigration-judge-san-francisco-immigration-court/'
#hon-print-maggard-immigration-judge-san-francisco-immigration-court/
response3 = requests.get(url3)
response3

<Response [200]>

In [20]:
soup3 = BeautifulSoup(response3.text,'html.parser')

bio_test2 = soup3.findAll('p')

bio_test2[0:1]


[<p>Juge Print was appointed as a DCIJ in November 2015. Prior to that appointment as a DCIJ, he served as the Acting Chief Immigration Judge from July 2015 to March 2016, and as an Assistant Chieflmmigration Judge from January 2012 to July 2015. Judge Print also served as an Immigration Judge at the San Francisco Immigration Court from May 2009 to December 2011.</p>]

# Create the links to scrape for the bios

In [21]:

# url3= 'https://www.ratemyimmigrationjudge.com/listing/hon-print-maggard-immigration-judge-san-francisco-immigration-court/'
judge_city[0]
front_half_url = 'https://www.ratemyimmigrationjudge.com/listing/hon-'
middle_part_url = '-immigration-judge-'
last_part_url = '-immigration-court/'
for i in range(len(judge_city)):
    judge_part_url = judge_city[i][0]
    city_part_url = judge_city[i][1]
    full_url = front_half_url+judge_part_url+middle_part_url+city_part_url+last_part_url
    judge_city[i].append(full_url)

judge_city[0:5]




[['John-W-Cortes',
  'Eloy',
  'https://www.ratemyimmigrationjudge.com/listing/hon-John-W-Cortes-immigration-judge-Eloy-immigration-court/'],
 ['Jennifer-I-Gaz',
  'Eloy',
  'https://www.ratemyimmigrationjudge.com/listing/hon-Jennifer-I-Gaz-immigration-judge-Eloy-immigration-court/'],
 ['Paul-Habich',
  'Eloy',
  'https://www.ratemyimmigrationjudge.com/listing/hon-Paul-Habich-immigration-judge-Eloy-immigration-court/'],
 ['Jose-L-Penalosa-Jr',
  'Eloy',
  'https://www.ratemyimmigrationjudge.com/listing/hon-Jose-L-Penalosa-Jr-immigration-judge-Eloy-immigration-court/'],
 ['Marni-Guerrero',
  'Florence',
  'https://www.ratemyimmigrationjudge.com/listing/hon-Marni-Guerrero-immigration-judge-Florence-immigration-court/']]

In [23]:
num_error = 0
num_success = 0

for i in range(len(judge_city)):
    try:
        response_it = requests.get(judge_city[i][2])
    except:
        judge_city[i].append(response_it)
    else:
        soup_it = BeautifulSoup(response_it.text,'html.parser')
        bio_test2 = soup_it.findAll('p')
        bio = bio_test2[0:1]
        judge_city[i].append(bio)



Success for  John-W-Cortes Total Hit 1
Success for  Jennifer-I-Gaz Total Hit 2
Success for  Paul-Habich Total Hit 3
Success for  Jose-L-Penalosa-Jr Total Hit 4
Success for  Marni-Guerrero Total Hit 5
Success for  Bruce-A-Taylor Total Hit 6
Success for  Robert-C-Bartlemay-Sr Total Hit 7
Success for  Elizabeth-A-Cottor Total Hit 8
Success for  LaMonte-S-Freerks Total Hit 9
Success for  Joseph-S-Imburgia Total Hit 10
Success for  Ken-Josephson Total Hit 11
Success for  Melissa-B-Karlen Total Hit 12
Success for  Munish-Sharda Total Hit 13
Success for  Linda-Spencer-Walters Total Hit 14
Success for  John-W-Davis Total Hit 15
Success for  Kathryn-L-DeAngelis Total Hit 16
Success for  Sean-H-Keenan Total Hit 17
Success for  Lauren-Mathon Total Hit 18
Success for  Bryan-E-DePowell Total Hit 19
Success for  Ravit-R-Halperin Total Hit 20
Success for  Steven-Marcus Total Hit 21
Success for  Nathaniel-B-Walker Total Hit 22
Success for  Curtis-G-White Total Hit 23
Success for  Jeffrey-V-Muñoz Total

In [25]:
judge_city[3:10]

[['Jose-L-Penalosa-Jr',
  'Eloy',
  'https://www.ratemyimmigrationjudge.com/listing/hon-Jose-L-Penalosa-Jr-immigration-judge-Eloy-immigration-court/',
  [<p>It seems we can’t find what you’re looking for. Perhaps searching can help.</p>]],
 ['Marni-Guerrero',
  'Florence',
  'https://www.ratemyimmigrationjudge.com/listing/hon-Marni-Guerrero-immigration-judge-Florence-immigration-court/',
  [<p>Attorney General Jeff Sessions appointed Marni Guerrero to begin hearing cases in October 2018. Judge Guerrero earned a Bachelor of Science in 1999 from Arizona State University, a Juris Doctor in 2003 from the Arizona State University Sandra Day O’Connor College of Law, and a Master of International Affairs in 2010 from the Columbia University School of International and Public Affairs. From 2017 to 2018, she was an assistant chief counsel with Immigration and Customs Enforcement, Department of Homeland Security. From 2013 to 2017, she was in private practice. From 2010 to 2013, she served as an